In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from testing import tester,display_stats,avg_stats,save_stats
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from tqdm import tqdm as pbar
from torch.distributions.normal import Normal
from utils.data_utils import data_helper,uci_helper

from models.deep_ensemble import Ensemble 
from trainers.deep_ensemble_trainer import Ensemble_Trainer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
def train_and_test(data,input_dim,num_units=128,batch_size = 512,qr_reg = False, 
              K=0.0,epochs = 48 ,lr =1e-2, n_splits=5,spacing=64):
    
    """
          does k-fold splitting w
    
    """
    

    kf = KFold(n_splits = n_splits)
    output_dim = 1
    dataset_stats = []
    iso_nll_inf_count = 0
    max_fx_up = -1
    
    
    for i,idx in enumerate(kf.split(data)):
        train_index, test_index = idx
        helper       = data_helper(data,train_index,test_index,input_dim)
        train_loader = helper.train_loader 
        test_loader  = helper.test_loader
        
        y_mean = helper.y_mean
        y_std  = helper.y_std
        x_range = helper.x_range


        ensemble      = Ensemble(input_dim =input_dim , output_dim=1, units= num_units)
        trainer       = Ensemble_Trainer(model = ensemble,input_dim =input_dim,
                                batch_size=batch_size,lr = lr,
                                device = device,mean=y_mean,std=y_std,
                                qr_reg = qr_reg,K =K,x_range = x_range)
        
        
        time                      = trainer.train(train_loader,epochs)
        iso,delta,t_cdf,iso_time  = trainer.fit_isotonic(train_loader) 
        test_util                 = tester(network = ensemble,delta=delta,iso=iso,
                                      t_cdf=t_cdf,mean=y_mean,std=y_std,
                                      qr_reg =qr_reg,K=K)
        
        current_split_stats,fx_up,count       = test_util.test(test_loader)
        iso_nll_inf_count = max(count,iso_nll_inf_count)
        
        current_fx_up = torch.max(fx_up)
        max_fx_up = max(current_fx_up.item(),max_fx_up)
        
       
        current_split_stats.append(time)
        current_split_stats.append(iso_time)
        dataset_stats.append(current_split_stats)
                
    
    dataset_stats = avg_stats(dataset_stats)
    return dataset_stats,max_fx_up,iso_nll_inf_count
        
        
    

In [3]:
def multiple_runs(name,qr=False,K=0.0,times=5):
    
    data = uci_helper(name)
    epochs       = 64
    units        = 128
    learn_rate   = 1e-2
    input_dim    = data.shape[1]-1
    batch_size   = 512

    overall_stats = []
    max_iso_nll_inf_count = -1
    max_fx_up = -1
    for i in pbar(range(times)):
        dataset_stats,fx_up,count  = train_and_test(data,input_dim,units,batch_size,epochs=epochs,
                                       lr=learn_rate,qr_reg = qr,K=K)
        
        overall_stats.append(dataset_stats)
        max_iso_nll_inf_count = max(max_iso_nll_inf_count,count)
        max_fx_up = max(max_fx_up,fx_up)
    
    save_stats(overall_stats,name,qr,K)
    print("iso nll count : {} max likelihood {}".format(max_iso_nll_inf_count,max_fx_up))

In [4]:
multiple_runs("airfoil", qr= False,K=0.0)

100%|██████████| 5/5 [04:24<00:00, 52.90s/it]

calib         : 25.52 -+ 1.90
iso_calib     : 40.42 -+ 2.14
rmse          : 3.17 -+ 0.04
iso_rmse      : 3.16 -+ 0.05
nll           : 3.02 -+ 0.08
iso_nll       : -1.53 -+ 0.61
time          : 10.36 -+ 0.10
iso_time      : 0.02 -+ 0.00
iso nll count : 71 max likelihood 12083.046875


In [5]:
multiple_runs("airfoil", qr=True,K=4.0)

100%|██████████| 5/5 [05:32<00:00, 66.46s/it]

calib         : 20.96 -+ 3.40
iso_calib     : 24.39 -+ 3.52
rmse          : 3.31 -+ 0.04
iso_rmse      : 3.29 -+ 0.04
nll           : 2.82 -+ 0.04
iso_nll       : -1.08 -+ 0.43
time          : 13.19 -+ 0.09
iso_time      : 0.02 -+ 0.00
iso nll count : 46 max likelihood 6748.892578125
